In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import spacy
# nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim.downloader
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer

import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, optimizers, layers
from keras.preprocessing import text, sequence
from keras.utils import plot_model

!pip install -q -U keras-tuner
import kerastuner as kt

import IPython


from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import re

from nltk import pos_tag
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer 

from gensim.models import word2vec

from IPython.display import display 

import seaborn as sns
import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

sns.set(context = 'notebook', style = 'whitegrid')
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows',50)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\caell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\caell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\caell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [134]:
df = pd.read_csv('../data/processed_text.csv')
X=df.Text
y=df.Grade

Below text thanks to:https://www.kaggle.com/jjjohnson/experiments-sentiment-analysis-raw-data-gru-lstm/edit

https://towardsdatascience.com/hands-on-nlp-deep-learning-model-preparation-in-tensorflow-2-x-2e8c9f3c7633

GloVe embeddigns thanks to Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]




Thanks to https://keras.io/examples/nlp/pretrained_word_embeddings/ for the below

In [ ]:

tokenizer = Tokenizer()

tokenizer.fit_on_texts(X)

word_index = tokenizer.word_index
total_vocabulary = word_index.keys()
num_words = len(total_vocabulary)
print(f'total vocabulary length: {num_words}')

glove = {}
with open('glove.840B.300d.txt',
          'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype = np.float32)
            glove[word] = vector


num_tokens = len(total_vocabulary) + 2
embedding_dim = 300
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = glove.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits +=1
    else:
        misses += 1
print(f'converted {hits} words ({misses} misses)')
    

In [132]:
tokens = tokenizer.texts_to_sequences(df.Text)
X = pad_sequences(tokens)


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)
X_t, X_val, y_t, y_val = train_test_split(X_train, y_train, random_state = 42, test_size = 0.2)

In [133]:
X_t

array([[   0,    0,    0, ...,  118,  100, 3490],
       [   0,    0,    0, ...,   72,    6,  234],
       [   0,    0,    0, ...,   64,   68,   54],
       ...,
       [   0,    0,    0, ...,    1, 9379,  155],
       [   0,    0,    0, ...,    1, 3530,  136],
       [   0,    0,    0, ..., 2126, 1034, 2127]])

In [135]:
embedding_layer=Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False)

In [136]:
model = Sequential()
model.add(Input(shape=(X_t_glove.shape[1])))
model.add(embedding_layer)
model.add(LSTM(1, activation='relu',
               return_sequences = True))
model.add(Dense(1, activation='relu'))

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])
model.summary()



Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 3095, 300)         3561300   
_________________________________________________________________
lstm_12 (LSTM)               (None, 3095, 1)           1208      
_________________________________________________________________
dense_10 (Dense)             (None, 3095, 1)           2         
Total params: 3,562,510
Trainable params: 1,210
Non-trainable params: 3,561,300
_________________________________________________________________


In [137]:
X_val_glove.shape

(52, 1291)

In [138]:
model.fit(X_t, y_t,
         batch_size=10,
         epochs=10,
         validation_data=(X_val, y_val),
      
         )

Epoch 1/10
21/21 [==============================] - 13s 621ms/step - loss: 4.8098 - mae: 4.8098 - val_loss: 4.3462 - val_mae: 4.3462
Epoch 2/10
21/21 [==============================] - 12s 584ms/step - loss: 4.8098 - mae: 4.8098 - val_loss: 4.3462 - val_mae: 4.3462
Epoch 3/10
21/21 [==============================] - 13s 604ms/step - loss: 4.8098 - mae: 4.8098 - val_loss: 4.3462 - val_mae: 4.3462
Epoch 4/10
21/21 [==============================] - 13s 632ms/step - loss: 4.8098 - mae: 4.8098 - val_loss: 4.3462 - val_mae: 4.3462
Epoch 5/10
21/21 [==============================] - 13s 609ms/step - loss: 4.8098 - mae: 4.8098 - val_loss: 4.3462 - val_mae: 4.3462
Epoch 6/10
21/21 [==============================] - 16s 764ms/step - loss: 4.8098 - mae: 4.8098 - val_loss: 4.3462 - val_mae: 4.3462
Epoch 7/10
21/21 [==============================] - 16s 771ms/step - loss: 4.8098 - mae: 4.8098 - val_loss: 4.3462 - val_mae: 4.3462
Epoch 8/10
21/21 [==============================] - 16s 743ms/step - 